In [1]:
import pandas as pd
from glob import glob

In [21]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [28]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

pio.templates.default = "plotly_white"

In [185]:
list_df = []
for file in glob('./csv/*.csv')[:5]:
    list_df.append(pd.read_csv(file))
    
df = pd.concat(list_df, ignore_index=True)

In [186]:
df = df.pivot(index=['label','newspaper','year'], columns='topic',values='rel_bias').reset_index()
df.columns.name = None

In [135]:
df.to_csv('./csv/bias_all_variables.csv', index=False)

In [188]:
df = pd.read_csv('./csv/bias_all_variables.csv')

In [189]:
pca = PCA()
scaler = StandardScaler()

In [190]:
data = scaler.fit_transform(df.iloc[:,3:])

In [191]:
pca.fit(data)

PCA()

In [192]:
fig = px.bar(x=range(1,pca.n_components_+1),
             y=pca.explained_variance_ratio_,
             labels={'x':'Principal Component','y':'Explained Variance Ratio'})
fig.update_layout(showlegend=False)

In [193]:
axe1 = 1
axe2 = 2

df_pca = pd.DataFrame(pca.transform(data), columns=range(1,pca.n_components_+1))
df_pca = pd.concat([df_pca, df.iloc[:,:3]],axis=1)

var_x = pca.components_[axe1-1,:].tolist()
var_x += [-x for x in var_x]
var_y = pca.components_[axe2-1,:].tolist()
var_y += [-y for y in var_y]
var_labels = ['Etat:autorité','immigration:choisie','impots:pression','lgbt:lobby','travail:effort',
              'Etat:usagers','immigration:vie','impots:évasion','lgbt:agression','travail:exploitation']


fig = px.scatter(df_pca, x=axe1, y=axe2, color='newspaper',text='year',
                 labels={str(axe1):f"PC {axe1}", str(axe2):f"PC {axe2}", 'newspaper':'Newspaper'})

fig.add_trace(go.Scatter(
    x=var_x,
    y=var_y,
    mode="text",     
    name="",
    text=var_labels
))

fig.show()

In [194]:
px.line(df_pca, x='year',y=1,color='newspaper')

In [195]:
px.line(df_pca, x='year',y=2,color='newspaper')

In [196]:
px.line(df_pca, x='year',y=3,color='newspaper')